# Miscellaneous plots

This notebook contains miscellaneous plots / analysis. See the headers for more detail

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

import xarray as xr
import pandas as pd
from scipy import sparse

import sys
sys.path.append('../Climate-Learning/')

import general_purpose.uplotlib as uplt
import general_purpose.cartopy_plots as cplt
import general_purpose.utilities as ut

HOME = './'

## Visualize the data normalization procedure

In [ ]:
lon = np.load('common/lon.npy')
lat = np.load('common/lat.npy')
LON, LAT = np.meshgrid(lon,lat)

y = 'r800y' # use the networks trained on 800 years of data

X_means = np.stack([np.load(f'common/{y}/fold_{fold}/X_mean.npy') for fold in range(5)])
X_stds = np.stack([np.load(f'common/{y}/fold_{fold}/X_std.npy') for fold in range(5)])

X_stds[X_stds == 1] = np.nan

X_means.shape, X_stds.shape

In [ ]:
mfp_kwargs = dict(one_fig_layout=120, figsize=(10,5),
                  projections=[cplt.ccrs.Orthographic(central_latitude=90), cplt.ccrs.PlateCarree()],
                  extents=[None, (-5, 10, 39, 55)],
                  titles=[r'Geopotential height [m]', 'Soil moisture [$m^3/m^3$]'],
                 )
cmaps=['RdBu_r', 'BrBG']

std_kwargs = dict(cmaps=['Reds', 'Greens'], vmin=0, colorbar='disabled')

In [ ]:
for fold in range(5):
    _ = cplt.mfp(LON,LAT,X_means[fold], **mfp_kwargs, fig_num=8+fold, cmaps=cmaps)
    fig = _[0].get_figure()
    fig.suptitle(f'fold {fold}')

Since the fields are already anomalies, the means are very close to 0

In [ ]:
for fold in range(5):
    _ = cplt.mfp(LON,LAT,X_stds[fold], **mfp_kwargs, fig_num=8+fold, **std_kwargs)
    fig = _[0].get_figure()
    fig.suptitle(f'fold {fold}')

In [ ]:
_ = cplt.mfp(LON,LAT,X_stds.mean(axis=0), **mfp_kwargs, fig_num=1, **std_kwargs)
fig = _[0].get_figure()
fig.suptitle(f'Mean of stds')
_ = cplt.mfp(LON,LAT,X_stds.std(axis=0), **mfp_kwargs, fig_num=2, cmaps=['Reds', 'Greens'], vmin=0, colorbar='disabled')
fig = _[0].get_figure()
fig.suptitle(f'Std of stds')

The maps of std across the folds are extremely similar, we can show only one

In [ ]:
fold = 0
_ = cplt.mfp(LON,LAT,X_stds[fold], **mfp_kwargs, fig_num=8+fold, **std_kwargs)
fig = _[0].get_figure()
fig.suptitle(f'pixel-wise standard deviation')

# fig.tight_layout(w_pad=0)
fig.tight_layout()

fig.savefig(f'{HOME}/STD.pdf')
# fig.savefig(f'{HOME}/STD.png', dpi=300)